In [10]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, PReLU,BatchNormalization, Flatten, UpSampling2D, LeakyReLU, Dense, Input, add
from tensorflow.keras.applications import VGG19
from tqdm import tqdm
from PIL import Image
import random
from scipy.ndimage import zoom

In [2]:
lr_images_path = '../data/lr_images'
hr_images_path = '../data/hr_images'

In [3]:
lr_filenames = sorted(os.listdir(lr_images_path))[:]
hr_filenames = sorted(os.listdir(hr_images_path))[:]

# Verify that the filenames match
if lr_filenames != hr_filenames:
    raise ValueError("Filenames in lr_images and hr_images do not match!")

In [4]:
lr_images = []
hr_images = []


for img_name in lr_filenames:
    lr_img = Image.open(os.path.join(lr_images_path, img_name))
    hr_img = Image.open(os.path.join(hr_images_path, img_name))
    
    # Convert grayscale images to RGB format
    if lr_img.mode != 'RGB':
        lr_img = lr_img.convert('RGB')
        
    if hr_img.mode != 'RGB':
        hr_img = hr_img.convert('RGB')
    
    lr_img = np.asarray(lr_img)
    hr_img = np.asarray(hr_img)
    
    lr_images.append(lr_img)
    hr_images.append(hr_img)

In [5]:
lr_images_array = np.stack(lr_images)
hr_images_array = np.stack(hr_images)

print("LR Images Array Shape:", lr_images_array.shape)
print("HR Images Array Shape:", hr_images_array.shape)

LR Images Array Shape: (15129, 64, 64, 3)
HR Images Array Shape: (15129, 128, 128, 3)


In [8]:
# Split the data into training, validation, and test sets
random_seed = 42 

train_lr_images, val_test_lr_images, train_hr_images, val_test_hr_images = train_test_split(
    lr_images_array, hr_images_array, test_size=0.3, random_state=random_seed
)

val_lr_images, test_lr_images, val_hr_images, test_hr_images = train_test_split(
    val_test_lr_images, val_test_hr_images, test_size=0.5, random_state=random_seed
)

In [6]:
generator = load_model('results/generator_ckpts/gen_e_8.h5')

2023-09-20 02:51:49.588430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-20 02:51:49.589818: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [9]:
srgan_hr_predictions = generator.predict(test_lr_images)

2023-09-20 02:52:37.118048: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-20 02:52:38.615523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - 76s 1s/step


In [14]:
def upscale_bicubic(images, scale_factor = 2):
    # Upscale using bicubic interpolation
    upscaled_images = zoom(images, zoom=(1, scale_factor, scale_factor, 1), order=3)
    return upscaled_images

In [15]:
def upscale_nearest(images, scale_factor = 2):
    # Upscale using nearest-neighbor interpolation
    upscaled_images = zoom(images, zoom=(1, scale_factor, scale_factor, 1), order=0)
    return upscaled_images

In [16]:
bicubic_interpolation = upscale_bicubic(test_lr_images)

nearest_interpolation = upscale_nearest(test_lr_images)

In [17]:
type(bicubic_interpolation)

numpy.ndarray